In [1]:
import sys

sys.path.append('/home/alta/Conversational/OET/al826/2022/DA_classification/1.2-encoder_decoder')

In [2]:
from src.handlers.eval_handler import EvalHandler, EnsembleEvaluator
from src.helpers.conv_handler import ConvHandler

eval_path = 'swda/standard/test.json'
#E = EvalHandler('baseline/2e-5/1')
#probs, labels = E.predictions(eval_path=eval_path)

Ens = EnsembleEvaluator('baseline/1e-5')
probs, labels = Ens.predictions(eval_path=eval_path)

#Ens = EnsembleEvaluator('baseline/1e-5)

100%|██████████| 19/19 [00:00<00:00, 33.09it/s]


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [3]:
from src.config import config 
label_names = ConvHandler.load_label_info(f'{config.base_dir}/data/{eval_path}')

In [4]:
from sklearn.calibration import calibration_curve
from sklearn import metrics

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from collections import Counter
import numpy as np
import time

class MutliClassEval:
    @classmethod
    def ensemble(cls, ensemble, labels, detail=False, names=None):
        seed_probs = [np.concatenate(seed_probs, axis=0) for seed_probs in ensemble]
        mean_probs = np.mean(seed_probs, axis=0)
        flat_labels = np.concatenate(labels, axis=0)
        cls.acc_report(mean_probs, flat_labels)

    @classmethod
    def seed_eval(cls, ensemble, labels, detail=False, names=None):
        for seed in ensemble:
            cls.seq_preds(seed, labels, detail, names)

    @classmethod
    def seq_preds(cls, probs, labels, detail=False, names=None):
        flat_probs  = np.concatenate(probs, axis=0)
        flat_labels = np.concatenate(labels, axis=0)
        cls.acc_report(flat_probs, flat_labels)
        
    @staticmethod
    def acc_report(probs, labels, detail=False, names=None):
        decision = np.argmax(probs, axis=-1)
        
        if (names and detail):
            decision = [names[i] for i in decision]
            labels = [names[i] for i in labels]

        report = metrics.classification_report(labels, decision, digits=3)  
        
        rows = report.split('\n')
        header = rows[0]
        summary = '\n'.join(rows[-4:-1])
        
        print(header)
        if detail:
            #This is all done to be able to sort by F1 (can change index to sort by P,R or support)
            report = metrics.classification_report(labels, decision, digits=3, output_dict=True)  
            results = [[k, v['precision'], v['recall'], v['f1-score'], v['support']] \
                       for k, v in report.items() if k not in ['weighted avg', 'macro avg', 'accuracy']] 
            results.sort(key=lambda x: x[3], reverse=True)
            
            width=max([len(x[0]) for x in results])
            for row in results:
                print(f' {row[0]:>{width}} {row[1]:>9.3f} {row[2]:>9.3f} {row[3]:>9.3f} {row[4]:>9}')
            print()
        print(summary)
        
MutliClassEval.seed_eval(probs, labels)

              precision    recall  f1-score   support
    accuracy                          0.811      4514
   macro avg      0.640     0.624     0.611      4514
weighted avg      0.812     0.811     0.809      4514
              precision    recall  f1-score   support
    accuracy                          0.805      4514
   macro avg      0.601     0.564     0.570      4514
weighted avg      0.802     0.805     0.801      4514
              precision    recall  f1-score   support
    accuracy                          0.809      4514
   macro avg      0.617     0.599     0.598      4514
weighted avg      0.804     0.809     0.804      4514


/home/miproj/4thyr.oct2019/al826/anaconda3/envs/torch1.7/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/miproj/4thyr.oct2019/al826/anaconda3/envs/torch1.7/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
Ens = EnsembleEvaluator('baseline/2e-5')
probs, labels = E.predictions(eval_path='swda/standard/test.json')


NameError: name 'E' is not defined

In [ ]:
from types import SimpleNamespace

eval_path = f"swda/standard/test.json"

test_args = {'eval_path':eval_path,
             'bsz':2,
             'lim':None}

test_args = SimpleNamespace(**test_args)
E.evaluate(test_args)

#words, scores = E.saliency(test_args, N=100, utt_num=63)
